In [ ]:
# First, install required packages
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

packages = ["streamlit", "pyngrok", "PyPDF2", "gtts", "resemble"]
for package in packages:
    try:
        install(package)
    except:
        pass

# Now create the app.py file
with open('app.py', 'w') as f:
    f.write('''
import streamlit as st
import PyPDF2
from gtts import gTTS
import io
import base64
from datetime import datetime
from resemble import Resemble
import tempfile
import os
import time

# Page configuration
st.set_page_config(
    page_title="Audiobook Generator Pro",
    page_icon="📚",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for styling
st.markdown("""
<style>
    .main-header {
        font-size: 3rem;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 2rem;
    }
    .sub-header {
        font-size: 1.5rem;
        color: #ff7f0e;
        margin-bottom: 1rem;
    }
    .success-box {
        background-color: #d4edda;
        color: #155724;
        padding: 15px;
        border-radius: 5px;
        margin: 10px 0;
    }
    .info-box {
        background-color: #d1ecf1;
        color: #0c5460;
        padding: 15px;
        border-radius: 5px;
        margin: 10px 0;
    }
    .warning-box {
        background-color: #fff3cd;
        color: #856404;
        padding: 15px;
        border-radius: 5px;
        margin: 10px 0;
    }
    .button-primary {
        background-color: #1f77b4;
        color: white;
        border: none;
        padding: 10px 20px;
        border-radius: 5px;
        font-size: 16px;
        cursor: pointer;
    }
    .button-secondary {
        background-color: #ff7f0e;
        color: white;
        border: none;
        padding: 10px 20px;
        border-radius: 5px;
        font-size: 16px;
        cursor: pointer;
    }
    .button-primary:hover {
        background-color: #16609c;
    }
    .button-secondary:hover {
        background-color: #e67300;
    }
    .footer {
        text-align: center;
        margin-top: 2rem;
        color: #6c757d;
    }
    .audio-container {
        background-color: #f8f9fa;
        padding: 20px;
        border-radius: 10px;
        margin: 15px 0;
    }
</style>
""", unsafe_allow_html=True)

# App title and description
st.markdown('<h1 class="main-header">🎧 Audiobook Generator Pro</h1>', unsafe_allow_html=True)
st.markdown("""
Convert your PDF documents into audio books with just a few clicks.
Upload your PDF file, extract text, and transform it into spoken audio using different voice options.
""")

# Initialize Resemble API (if available)
resemble_available = False
try:
    Resemble.api_key("3nAflVLgNDzxQQzEiielDAtt")
    project_uuid = Resemble.v2.projects.all(1, 10)['items'][0]['uuid']
    voice_uuid = Resemble.v2.voices.all(1, 10)['items'][0]['uuid']
    resemble_available = True
except Exception as e:
    st.sidebar.warning(f"Resemble API not configured. Using gTTS as fallback. Error: {str(e)}")

# Sidebar
with st.sidebar:
    st.title("Settings")

    # Language selection
    language = st.selectbox(
        "Select Language:",
        ["en", "es", "fr", "de", "hi"],
        index=0
    )

    # Speed selection
    speed = st.radio(
        "Speech Speed:",
        ["Normal", "Slow"],
        index=0
    )

    st.markdown("---")
    st.info("""
    **How it works:**
    1. Upload a PDF file
    2. Extract text from the PDF
    3. Generate audio using gTTS
    4. Convert to your voice using Resemble AI
    """)

# Helper functions
def extract_text_from_pdf(pdf_file):
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text() + " "
    return text

def text_to_speech(text, language='en', slow=False):
    cleaned_text = " ".join(text.split())
    tts = gTTS(text=cleaned_text, lang=language, slow=slow)
    audio_buffer = io.BytesIO()
    tts.write_to_fp(audio_buffer)
    audio_buffer.seek(0)
    return audio_buffer

def resemble_text_to_speech(text, language='en'):
    try:
        # Create a clip using Resemble API
        response = Resemble.v2.clips.create_sync(
            project_uuid,
            voice_uuid,
            text[:5000],  # Limit text length for demo
            title=f"Audiobook_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            sample_rate=22050,
            output_format="mp3",
            precision="PCM_16",
            include_timestamps=False
        )

        if response['success']:
            # Download the audio
            clip_url = response['item']['audio_src']
            import requests
            audio_response = requests.get(clip_url)
            audio_buffer = io.BytesIO(audio_response.content)
            audio_buffer.seek(0)
            return audio_buffer
        else:
            st.error("Resemble API Error: " + response.get('message', 'Unknown error'))
            return None
    except Exception as e:
        st.error(f"Error with Resemble API: {str(e)}")
        return None

# Initialize session state variables
if 'extracted_text' not in st.session_state:
    st.session_state.extracted_text = ""
if 'gtts_audio' not in st.session_state:
    st.session_state.gtts_audio = None
if 'resemble_audio' not in st.session_state:
    st.session_state.resemble_audio = None

# Main content area
uploaded_file = st.file_uploader("Upload a PDF file", type=["pdf"])

if uploaded_file:
    # Extract text from PDF
    if st.button("Extract Text from PDF") or st.session_state.extracted_text:
        with st.spinner("Extracting text from PDF..."):
            if not st.session_state.extracted_text:
                st.session_state.extracted_text = extract_text_from_pdf(uploaded_file)

        st.markdown(f'<div class="success-box">✅ Successfully extracted {len(st.session_state.extracted_text)} characters</div>', unsafe_allow_html=True)

        # Text preview
        st.markdown("### 📝 Extracted Text Preview")
        st.text_area("Transcribed Text",
                    st.session_state.extracted_text[:1000] + "..." if len(st.session_state.extracted_text) > 1000 else st.session_state.extracted_text,
                    height=300,
                    key="extracted_text_area")

        # Step 1: Generate audio with gTTS
        st.markdown("---")
        st.markdown("### Step 1: Generate Audio with gTTS")

        if st.button("Generate Audio with gTTS", key="gtts_btn"):
            with st.spinner("Generating audio with gTTS..."):
                st.session_state.gtts_audio = text_to_speech(
                    st.session_state.extracted_text,
                    language=language,
                    slow=(speed == "Slow")
                )

            if st.session_state.gtts_audio:
                st.markdown('<div class="success-box">✅ gTTS audio generated successfully!</div>', unsafe_allow_html=True)

        # Display gTTS audio if available
        if st.session_state.gtts_audio:
            st.markdown('<div class="audio-container">', unsafe_allow_html=True)
            st.markdown("#### gTTS Audio")

            # Save audio to a temporary file
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp_file:
                tmp_file.write(st.session_state.gtts_audio.getvalue())
                tmp_file_path = tmp_file.name

            # Display audio player
            st.audio(tmp_file_path, format="audio/mp3")

            # Create download link
            with open(tmp_file_path, "rb") as file:
                btn = st.download_button(
                    label="Download gTTS Audio",
                    data=file,
                    file_name=f"gtts_audiobook_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp3",
                    mime="audio/mpeg",
                    key="gtts_download"
                )

            st.markdown('</div>', unsafe_allow_html=True)

            # Clean up temporary file
            os.unlink(tmp_file_path)

            # Step 2: Convert to Resemble AI voice
            st.markdown("---")
            st.markdown("### Step 2: Convert to Your Voice with Resemble AI")

            if resemble_available:
                if st.button("Convert to Your Voice", key="resemble_btn"):
                    with st.spinner("Converting to your voice with Resemble AI..."):
                        st.session_state.resemble_audio = resemble_text_to_speech(st.session_state.extracted_text, language=language)

                    if st.session_state.resemble_audio:
                        st.markdown('<div class="success-box">✅ Resemble AI audio generated successfully!</div>', unsafe_allow_html=True)
            else:
                st.markdown('<div class="warning-box">Resemble AI is not available. Please check your API configuration.</div>', unsafe_allow_html=True)

            # Display Resemble audio if available
            if st.session_state.resemble_audio:
                st.markdown('<div class="audio-container">', unsafe_allow_html=True)
                st.markdown("#### Resemble AI Audio (Your Voice)")

                # Save audio to a temporary file
                with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp_file:
                    tmp_file.write(st.session_state.resemble_audio.getvalue())
                    tmp_file_path = tmp_file.name

                # Display audio player
                st.audio(tmp_file_path, format="audio/mp3")

                # Create download link
                with open(tmp_file_path, "rb") as file:
                    btn = st.download_button(
                        label="Download Resemble Audio",
                        data=file,
                        file_name=f"resemble_audiobook_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp3",
                        mime="audio/mpeg",
                        key="resemble_download"
                    )

                st.markdown('</div>', unsafe_allow_html=True)

                # Clean up temporary file
                os.unlink(tmp_file_path)

# Footer
st.markdown("---")
st.markdown('<div class="footer">Audiobook Generator Pro • Created with Streamlit</div>', unsafe_allow_html=True)
''')

print("app.py created successfully!")

# Now run the Streamlit app with ngrok
from pyngrok import ngrok

# Set up ngrok
NGROK_AUTH_TOKEN = "2vfpQ1XvW86YZ3Bpf1haMZ8SwvO_76stJUyFhxnV81UjG7Gy4"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start tunnel (http protocol)
public_url = ngrok.connect(8501, "http")
print("🌐 Streamlit App URL:", public_url)

# Run Streamlit app
import subprocess
import os

# Set the PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION environment variable
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# Run Streamlit in the background
subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"])

app.py created successfully!
🌐 Streamlit App URL: NgrokTunnel: "https://28732273725a.ngrok-free.app" -> "http://localhost:8501"


<Popen: returncode: None args: ['streamlit', 'run', 'app.py', '--server.port...>